# DuckDB Notebook — Unsolved


## 0. Setup & Data

This notebook is **SQL-first**:
- you will build an end-to-end pipeline using **CTEs**
- you will run **data quality checks**
- you will produce a **BI-ready table**
- you will export results to Parquet

We generate:
- `products.parquet` (dimension)
- `sales.parquet` (facts)

We use `timestamp` (not ts).


In [ ]:
import duckdb
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

con = duckdb.connect()

In [ ]:
N_SALES = 300_000
N_PRODUCTS = 2_000

cities = ["Valencia", "Madrid", "Sevilla", "Bilbao", "Barcelona"]
categories = ["Electronics", "Fashion", "Home", "Sports", "Toys"]

In [ ]:
products = pd.DataFrame({
    "product_id": np.arange(1, N_PRODUCTS + 1),
    "category": [categories[i % len(categories)] for i in range(N_PRODUCTS)],
    "is_discontinued": (np.arange(1, N_PRODUCTS + 1) % 40 == 0),
})

sales = pd.DataFrame({
    "sale_id": np.arange(1, N_SALES + 1),
    "product_id": (np.arange(1, N_SALES + 1) * 37 % (N_PRODUCTS + 200)) + 1,
    "user_id": (np.arange(1, N_SALES + 1) * 13) % 50_000,
    "units": (np.arange(1, N_SALES + 1) * 7 % 5) + 1,
    "city": [cities[i % len(cities)] for i in range(N_SALES)],
    "has_discount": (np.arange(1, N_SALES + 1) % 10 == 0),
    "timestamp": [datetime(2025, 1, 1) + timedelta(seconds=i) for i in range(N_SALES)],
})
sales.loc[sales.index % 5000 == 0, "sale_id"] = 1
sales["gross_value"] = sales["units"] * ((sales["sale_id"] % 200) + 5)


In [ ]:
products.to_parquet("products.parquet", index=False)
sales.to_parquet("sales.parquet", index=False)

In [ ]:
con.execute("SELECT * FROM products.parquet LIMIT 5").df()

,product_id,category,is_discontinued
0,1,Electronics,False
1,2,Fashion,False
2,3,Home,False
3,4,Sports,False
4,5,Toys,False


In [ ]:
con.execute("SELECT * FROM sales.parquet LIMIT 5").df()

,sale_id,product_id,user_id,units,city,has_discount,timestamp,gross_value
0,1,38,13,3,Valencia,False,2025-01-01 00:00:00,18
1,2,75,26,5,Madrid,False,2025-01-01 00:00:01,35
2,3,112,39,2,Sevilla,False,2025-01-01 00:00:02,16
3,4,149,52,4,Bilbao,False,2025-01-01 00:00:03,36
4,5,186,65,1,Barcelona,False,2025-01-01 00:00:04,10


## Exercise 1 — Create a clean base layer (CTE 1)

Goal:
Build a base sales layer that:
- standardizes `city` (lowercase)
- computes `net_value` (apply 10% discount when has_discount=true)
- extracts `day` from timestamp

Return 10 rows.


In [ ]:
# TODO:
# - normalize city using LOWER
# - extract day from timestamp
# - compute net_value applying discount logic
# - keep original columns for reference

con.execute("""
SELECT
  sale_id,
  product_id,
  user_id,
  units,
  city,
  -- TODO: normalized city
  has_discount,
  timestamp,
  -- TODO: day column
  gross_value,
  -- TODO: net_value
FROM 'sales.parquet'
LIMIT 10
""").df()


,sale_id,product_id,user_id,units,city,city_norm,has_discount,timestamp,day,gross_value,net_value
0,1,38,13,3,Valencia,valencia,False,2025-01-01 00:00:00,2025-01-01,18,18.0
1,2,75,26,5,Madrid,madrid,False,2025-01-01 00:00:01,2025-01-01,35,35.0
2,3,112,39,2,Sevilla,sevilla,False,2025-01-01 00:00:02,2025-01-01,16,16.0
3,4,149,52,4,Bilbao,bilbao,False,2025-01-01 00:00:03,2025-01-01,36,36.0
4,5,186,65,1,Barcelona,barcelona,False,2025-01-01 00:00:04,2025-01-01,10,10.0
5,6,223,78,3,Valencia,valencia,False,2025-01-01 00:00:05,2025-01-01,33,33.0
6,7,260,91,5,Madrid,madrid,False,2025-01-01 00:00:06,2025-01-01,60,60.0
7,8,297,104,2,Sevilla,sevilla,False,2025-01-01 00:00:07,2025-01-01,26,26.0
8,9,334,117,4,Bilbao,bilbao,False,2025-01-01 00:00:08,2025-01-01,56,56.0
9,10,371,130,1,Barcelona,barcelona,True,2025-01-01 00:00:09,2025-01-01,15,13.5


## Exercise 2 — Data Quality: invalid product references

Goal:
Find sales rows whose product_id does NOT exist in products.

Deliver:
- a count of invalid rows
- and show 10 example rows (sale_id, product_id)

In [ ]:
# TODO:
# - detect sales with invalid product_id
# - use an ANTI JOIN between sales and products
# - return the total count of invalid rows

invalid_count = con.execute("""
SELECT
  -- TODO: count invalid sales
FROM 'sales.parquet' s
ANTI JOIN 'products.parquet' p
  ON -- TODO: join condition
""").df()


# TODO:
# - show a sample of invalid sales
# - select sale_id and product_id
# - limit to 10 rows

invalid_examples = con.execute("""
SELECT
  -- TODO: sale_id, product_id
FROM 'sales.parquet' s
ANTI JOIN 'products.parquet' p
  ON -- TODO: join condition
LIMIT 10
""").df()


In [ ]:
invalid_count

,invalid_sales
0,27270


In [ ]:
invalid_examples

,sale_id,product_id
0,55,2036
1,56,2073
2,57,2110
3,58,2147
4,59,2184
5,114,2019
6,115,2056
7,116,2093
8,117,2130
9,118,2167


## Exercise 3 — Data Quality: detect duplicates

Goal:
Simulate a real DQ check: "Do we have duplicate sale_id?"

Deliver:
- count of duplicated sale_id values
- list of top duplicated sale_id (if any)

Note:
In a real pipeline, this is a hard-fail check.


In [ ]:
# TODO (SQL):
# - GROUP BY sale_id
# - HAVING COUNT(*) > 1
# - COUNT how many sale_id are duplicated
# - Show sample duplicated ids


In [ ]:
# TODO (SQL):
# - group by sale_id
# - count how many times each sale_id appears
# - keep only sale_id with more than one occurrence
# - count how many duplicated sale_id exist

dup_summary = con.execute("""
WITH dups AS (
  SELECT
    -- TODO: sale_id
    -- TODO: count occurrences
  FROM 'sales.parquet'
  GROUP BY -- TODO
  HAVING -- TODO
)
SELECT
  -- TODO: count duplicated sale_id
FROM dups
""").df()


# TODO (SQL):
# - show example duplicated sale_id
# - order by number of occurrences desc
# - limit results

dup_examples = con.execute("""
SELECT
  -- TODO: sale_id
  -- TODO: count occurrences
FROM 'sales.parquet'
GROUP BY -- TODO
HAVING -- TODO
ORDER BY -- TODO
LIMIT 10
""").df()


In [ ]:
dup_summary

,duplicated_sale_ids
0,1


In [ ]:
dup_examples

,sale_id,n
0,1,60


## Exercise 4 — Enrichment

Build an enriched sales view that:
- joins sales with products to obtain the product category
- excludes discontinued products

Return a sample of 10 rows.


In [ ]:
# TODO:
# - join sales with products using product_id
# - exclude discontinued products
# - return a sample of rows

con.execute("""
SELECT
  s.sale_id,
  s.product_id,
  p.category,
  s.gross_value
FROM 'sales.parquet' s
INNER JOIN 'products.parquet' p
  ON -- TODO: join condition
WHERE -- TODO: filter discontinued products
LIMIT 10
""").df()


,sale_id,product_id,category,gross_value
0,1,38,Home,18
1,2,75,Toys,35
2,3,112,Fashion,16
3,4,149,Sports,36
4,5,186,Electronics,10
5,6,223,Home,33
6,7,260,Toys,60
7,8,297,Fashion,26
8,9,334,Sports,56
9,10,371,Electronics,15


## Exercise 5 — KPI Aggregation

Compute business KPIs by product category:
- number of orders
- total revenue

Only consider active (non-discontinued) products.


In [ ]:
# TODO:
# - join sales with products to get category
# - exclude discontinued products
# - aggregate by category
# - compute:
#   - number of orders
#   - total revenue
# - order results by revenue descending

con.execute("""
SELECT
  p.category,
  -- TODO: COUNT(*) AS orders
  -- TODO: SUM(...) AS revenue
FROM 'sales.parquet' s
INNER JOIN 'products.parquet' p
  ON -- TODO: join condition
WHERE -- TODO: filter active products
GROUP BY -- TODO: category
ORDER BY -- TODO: revenue desc
""").df()


,category,orders,revenue
0,Toys,47726,24680685.0
1,Sports,54546,23236576.0
2,Home,54546,16936503.0
3,Fashion,54546,11508916.0
4,Electronics,54546,5590825.0


## Exercise 6 — Data Quality Overview

Run the query below and interpret the results.

Focus on:
- which data quality checks are applied
- what each metric represents


In [ ]:
# TODO:
# - return total number of sales
# - return number of sales with invalid product_id
# - return number of sales linked to discontinued products

In [ ]:
# TODO:
# - return total number of sales rows
# - return number of sales with invalid product_id
# - return number of sales linked to discontinued products
#
# Hints:
# - COUNT(*) over the full sales table
# - ANTI JOIN to detect invalid references
# - INNER JOIN + filter for discontinued products

con.execute("""
SELECT
  -- TODO: total number of sales
  -- TODO: invalid product references
  -- TODO: discontinued product sales
""").df()


,total_sales,invalid_product_sales,discontinued_product_sales
0,300000,27270,6820


## Exercise 7 — Export Results

Export the KPI results by product category to a Parquet file.

This step simulates the output of an analytical pipeline.


In [ ]:
# TODO:
# - compute KPIs by category
# - export the result to a Parquet file
# - use COPY to write the output
# - choose a meaningful output file name

con.execute("""
COPY (
  SELECT
    p.category,
    -- TODO: COUNT(*) AS orders
    -- TODO: SUM(...) AS revenue
  FROM 'sales.parquet' s
  INNER JOIN 'products.parquet' p
    ON -- TODO: join condition
  WHERE -- TODO: filter active products
  GROUP BY -- TODO: category
) TO -- TODO: output file
(FORMAT PARQUET)
""")
